<a href="https://colab.research.google.com/github/sarkar-sayan/Feedback_Analysis-using-Machine_Learning_NLP_GenerativeAI/blob/main/POC_2_main(%20).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import re
import nltk
import os
#from langdetect import detect
#from googletrans import Translator
# Initialize Google Translator
#translator = Translator()

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
from groq import Groq
!export GROQ_API_KEY='YOUR_GROQ_API_KEY'

client = Groq(
    # This is the default and can be omitted
    api_key='YOUR_GROQ_API_KEY'
    #api_key=os.environ.get("GROQ_API_KEY"),
)

In [5]:
# Text preprocessing function
# Translate text if not in English
#def translate_text_if_needed(text):
#    try:
#        language = detect(text)
#        if language != 'en':
#            translated = translator.translate(text, dest='en')
#            return translated.text
#        return text
#    except Exception as e:
#        return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text


In [6]:
def prepare_data(dataset_url):
  df = pd.read_csv(dataset_url)
  # preprocess data accroding to survey form
  df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
  df = df.dropna()

  # Join all textual columns into one
  feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
  df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
  #df['All_feedback'] = df['All_feedback'].apply(translate_text_if_needed)
  df.drop(feedback_columns, axis=1, inplace=True)
  df.drop('Final Feedback', axis=1, inplace=True)
  # Preprocess textual feedback
  df['All_feedback'] = df['All_feedback'].apply(preprocess_text)
  return df

In [7]:
def train_test_model(data):
  X = data.drop('Final Label', axis=1)
  y = data['Final Label']

  # Train/Test Split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
  # Define ColumnTransformer to separate numerical and text data processing
  preprocessor = ColumnTransformer(
      transformers=[
          ('num', StandardScaler(), X.select_dtypes(include='number').columns.tolist()),
          ('text', TfidfVectorizer(), 'All_feedback')
          ],
      remainder='drop'
      )

  # Define the pipeline
  pipeline = Pipeline([
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression(random_state=42))
      ])

  pipeline.fit(X_train, y_train)

  # Predict on the test set
  y_pred = pipeline.predict(X_test)

  #print("Accuracy:", accuracy_score(y_test, y_pred))
  #print("Classification Report:")
  #print(classification_report(y_test, y_pred))
  return pipeline

In [8]:
def predict_feedback_proba(pipeline, sample):
    sample_df = pd.DataFrame([sample])
    sample_df = sample.to_frame().T
    y_prob = pipeline.predict_proba(sample_df)
    prob_pos = y_prob[0][1] * 100  # Probability of being positive
    prob_neg = y_prob[0][0] * 100  # Probability of being negative
    return prob_pos, prob_neg

In [9]:
# Function to generate detailed feedback using OpenAI's GPT-4
def generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg):
    # Convert numerical_data to a string representation
    numerical_data_str = str(numerical_data)

    prompt = f"Using the following numerical ratings and feedback:\n\nNumerical Ratings: {numerical_data_str}\n\nTextual Feedback: {textual_feedback}\n\nProbability of being positive: {prob_pos:.2f}%\n\nProbability of being negative: {prob_neg:.2f}%\n\nPlease provide a very short final feedback according to the client, i.e, either positive or negative. Not both. Give output in the form of either Positive, Mostly Positive, Negative or Mostly Negative with the probability percentage. And then Improvements or Matters to focus upon:"
    chat_completion = client.chat.completions.create(
    messages = [
    {
        "role": "system",
        "content": "you are a helpful assistant."
    },
{
        "role": "user",
        "content": prompt, # Use the formatted prompt here
    },
    ],
    model="llama3-8b-8192" # Add the model keyword argument here
    )
    return chat_completion.choices[0].message.content.strip()  # Extract the content from the response


In [10]:
def generate_feedback(pipeline, sample):
  # Example usage
  #prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
  #print(f"Sample:\n{sample}")
  #print(f"Probability of being positive: {prob_pos:.2f}%")
  #print(f"Probability of being negative: {prob_neg:.2f}%")

  numerical_data = sample.drop('All_feedback').to_dict()
  textual_feedback = sample['All_feedback']
  prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
  detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg)
  print(detailed_feedback)

In [11]:
def get_model_ready():
  dataset_url = input("Enter the dataset URL: ")
  df = prepare_data(dataset_url)
  pipeline = train_test_model(df)
  return pipeline

In [20]:
def get_feedback(pipeline):
  feedform_url = input("Enter the URL where feedback data for generating analysis is stored: ")
  data = prepare_data(feedform_url)
  for i in range(len(data)):
    sample = data.iloc[i]
    print("Feedback: ", i+1)
    generate_feedback(pipeline, sample)

In [13]:
def final_execute():
  pipeline = get_model_ready()
  get_feedback(pipeline)

In [15]:
def main():
  final_execute()

In [21]:
if __name__ == "__main__":
  main()

Enter the dataset URL: /content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form 1.csv
Enter the URL where feedback data for generating analysis is stored: /content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form 1.csv
Feedback:  1
Based on the provided numerical ratings and feedback, I would say that the final feedback is:

Positive (53.19%)

Improvements or Matters to focus upon:

* Improve response time and clarity in communication.
* Enhance understanding of business requirements by being more proactive and inquiring.
* Increase knowledge in technology used for faster resolution.
* Maintain high levels of courtesy, respect, and professionalism throughout interactions.
Feedback:  2
Based on the provided numerical ratings and feedback, I would classify the final feedback as:

**Mostly Positive** (86.09%)

Improvements or Matters to Focus Upon:

* Providing more clear and understandable information to clients (rated 2.0)
* Improving response time to client inquiri